# DIA + Synthetic Source Injection for AP
## Fake recovery metric plots -- NO Association

In [ ]:
# Parameters for time-square notebook run

collection = "u/sullii/DM-46495-Fakes"
repo = "/repo/main"
instrument = "LSSTComCam"

### Imports and butler repo settings

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy import units as u
from astropy import coordinates as coords
from astropy.table import vstack

import lsst.daf.butler as dafButler

In [ ]:
from lsst.utils.plotting import (
    get_multiband_plot_colors,
    get_multiband_plot_symbols,
    get_multiband_plot_linestyles,
)

clrs = get_multiband_plot_colors()

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

sns.set_context("talk")

In [ ]:
butler = dafButler.Butler(repo, collections=collection, instrument=instrument)

In [ ]:
drefs = butler.query_datasets(
    "fakes_goodSeeingDiff_matchDiaSrc", instrument=instrument, collections=collection
)

len(list(drefs))

#### Load tables and consolidate them

In [ ]:
catalogs = []
provenance_rows = []
diaSrcCats = []
rbscoreCats = []
candDiaSrcCats = []

for acat in drefs:
    matchcat = butler.get(acat)
    matchcat["visit"] = acat.dataId["visit"]
    matchcat["detector"] = acat.dataId["detector"]
    matchcat["band"] = acat.dataId["band"]
    matchcat["run"] = acat.run
    matchcat["day_obs"] = acat.dataId["day_obs"]
    catalogs.append(matchcat)

    dataIdrow = [v for k, v in acat.dataId.to_simple().dataId.items()]
    dataIdrow.append(acat.run)
    provenance_rows.append(dataIdrow)

    diaSrcCat = (
        butler.get(
            "fakes_goodSeeingDiff_diaSrc",
            visit=acat.dataId["visit"],
            detector=acat.dataId["detector"],
            band=acat.dataId["band"],
        )
        .asAstropy()
        .to_pandas()
    )

    diaSrcCat["run"] = acat.run
    diaSrcCat["visit"] = acat.dataId["visit"]
    diaSrcCat["detector"] = acat.dataId["detector"]
    diaSrcCat["band"] = acat.dataId["band"]
    diaSrcCats.append(diaSrcCat)

    candDiaSrcCat = (
        butler.get(
            "fakes_goodSeeingDiff_candidateDiaSrc",
            visit=acat.dataId["visit"],
            detector=acat.dataId["detector"],
            band=acat.dataId["band"],
        )
        .asAstropy()
        .to_pandas()
    )

    candDiaSrcCat["run"] = acat.run
    candDiaSrcCat["visit"] = acat.dataId["visit"]
    candDiaSrcCat["detector"] = acat.dataId["detector"]
    candDiaSrcCat["band"] = acat.dataId["band"]
    candDiaSrcCats.append(candDiaSrcCat)

    rbscoreCat = (
        butler.get(
            "fakes_goodSeeingRealBogusSources",
            visit=acat.dataId["visit"],
            detector=acat.dataId["detector"],
            band=acat.dataId["band"],
        )
        .asAstropy()
        .to_pandas()
    )
    rbscoreCat["run"] = acat.run
    rbscoreCat["visit"] = acat.dataId["visit"]
    rbscoreCat["detector"] = acat.dataId["detector"]
    rbscoreCat["band"] = acat.dataId["band"]
    rbscoreCats.append(rbscoreCat)

 Data provenance shows that there are three runs. 

In [ ]:
provenance = pd.DataFrame(
    provenance_rows,
    columns=["instrument", "detector", "visit", "band", "day_obs", "physFltr", "run"],
)

In [ ]:
diaSrcs = pd.concat(diaSrcCats)
matches = pd.concat(catalogs)
rbscores = pd.concat(rbscoreCats)
candDiaSrcs = pd.concat(candDiaSrcCats)

In [ ]:
# picking the last one only:
diaSrcs.set_index("id", inplace=True)
rbscores["diaSourceId"] = rbscores["id"]
rbscores.set_index("id", inplace=True)
found = matches.diaSourceId > 0
matches["found"] = matches.diaSourceId > 0
matches["isCandidate"] = matches.diaSourceId.isin(candDiaSrcs.id)
matches["dist_host"] = (
    np.sqrt(matches["delta_ra"] ** 2 + matches["delta_dec"] ** 2) * 3600
)
matches["delta_mag"] = matches["mag"] - matches["host_magnitude"]

#### Basic statistics on fake sources

In [ ]:
# print in nice format a summary of the table lengths and found fakes
print(
    f"""----------------------------------"
    "N diaSrcs          : {len(diaSrcs):6d}"
    "N candidateDiaSrcs : {len(candDiaSrcs):6d}"
    "N fakes            : {len(matches):6d}"
    " *  N found fakes       : {np.sum(matches["found"]):6d} in diaSrc"
    " *  N found fakes       : {np.sum(matches["isCandidate"]):6d} in candidateDiaSrc"
    " *  N Filtered fakes    : {np.sum(matches["found"]) - np.sum(matches["isCandidate"]):6d}"
    " *  N missed fakes      : {np.sum(~matches["found"]):6d}"
    """
)

In [ ]:
wid = 0.25
bins = np.arange(18, 28, wid)
plt.title("Magnitude distribution in all bands")
plt.hist(matches.mag, bins=bins, label="All", histtype="step", lw=2)
plt.hist(matches[found].mag, bins=bins, label="Found", alpha=0.5)
plt.xlabel("Fake mag")
plt.legend(loc="upper right")
plt.tight_layout()

In [ ]:
plt.title("SNR distribution")

plt.hist(
    matches.forced_base_PsfFlux_instFlux_SNR,
    bins=np.arange(0, 80, 1),
    label="All",
    histtype="step",
    lw=2,
)
plt.hist(
    matches[found].forced_base_PsfFlux_instFlux_SNR,
    bins=np.arange(0, 80, 1),
    label="Found",
    alpha=0.5,
)
plt.gca().set_yscale("log")
plt.xlabel("Fake SNR")
plt.legend(loc="upper right")
plt.tight_layout()

In [ ]:
bands = np.unique(matches["band"])

In [ ]:
# plotting grid size according to band length
nbands = len(bands)
nx = int(np.ceil(np.sqrt(nbands)))
ny = int(np.ceil(nbands / nx))
gridplots = (nx, ny)

In [ ]:
fig, axes = plt.subplots(*gridplots, figsize=(10, 8))
iax = 0

for flt in bands:
    ax = axes.flatten()[iax] if len(bands) > 1 else axes
    ax.set_title(f"Magnitudes distribution in LSSTCam {flt} filter")
    tab = matches[matches["band"] == flt]
    found = tab.diaSourceId > 0
    counts, _, _ = ax.hist(
        tab.mag,
        bins=bins,
        label="All",
        histtype="step",
        lw=2,
        color=clrs[flt],
    )
    counts_, bins_, _ = ax.hist(
        tab[found].mag,
        bins=bins,
        label="Found",
        alpha=0.5,
        color=clrs[flt],
    )
    iax += 1
    ax.set_xlabel("Fake mag")
plt.legend(loc="upper right")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10, 8))
plt.title("Positions of all lost fakes")
plt.scatter(
    matches[~found].x,
    matches[~found].y,
    c=matches[~found].mag,
    cmap="inferno",
    vmin=18,
    vmax=27,
    s=1000 / matches[~found].mag,
    alpha=0.75,
)
plt.colorbar(label="mag")
plt.title("Fakes lost position")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
hs = ax.hist2d(
    matches[~found].x,
    matches[~found].y,
    bins=(16, 16),
    cmap="Greys",
    alpha=0.5,
)
cb = plt.colorbar(hs[3], ax=ax)
cb.set_label("Counts per bin")
plt.title("Binned positions of all Fakes lost")
ax.set_aspect("equal")

In [ ]:
plt.hist(
    matches[found].dist_diaSrc, bins=30, histtype="step", lw=2, label="Found diaSrc"
)
plt.title("Distribution of recovered positions offsets")
plt.xlabel(r"$\Delta r$ [arcsec]")
plt.legend()
plt.show()

In [ ]:
dx = (
    matches[found].x
    - diaSrcs.loc[matches[found].diaSourceId, "base_SdssCentroid_x"].values
)
dy = (
    matches[found].y
    - diaSrcs.loc[matches[found].diaSourceId, "base_SdssCentroid_y"].values
)
plt.plot(dx, dy, ".", alpha=0.25)
plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
plt.grid()
plt.title("Scatter of recovered fakes X/Y positions", fontsize=10)
plt.xlabel(r"$\Delta x$ [px]", fontsize=14)
plt.ylabel(r"$\Delta y$ [px]", fontsize=14)
plt.gca().set_aspect("equal")
plt.tight_layout()

In [ ]:
dra = 3600 * (
    matches[found].ra
    - np.rad2deg(diaSrcs.loc[matches[found].diaSourceId, "coord_ra"].values)
)
ddec = 3600 * (
    matches[found].dec
    - np.rad2deg(diaSrcs.loc[matches[found].diaSourceId, "coord_dec"].values)
)
plt.plot(dra, ddec, ".", alpha=0.25)
plt.xlim(-0.6, 0.6)
plt.ylim(-0.6, 0.6)
plt.grid()
plt.title("Scatter of recovered fakes RA/Dec positions")
plt.xlabel(r"$\Delta RA$ [arcsec]", fontsize=14)
plt.ylabel(r"$\Delta Dec$ [arcsec]", fontsize=14)
plt.gca().set_aspect("equal")
plt.tight_layout()

In [ ]:
dra = 3600 * (
    matches[found].ra
    - np.rad2deg(diaSrcs.loc[matches[found].diaSourceId, "coord_ra"].values)
)
ddec = 3600 * (
    matches[found].dec
    - np.rad2deg(diaSrcs.loc[matches[found].diaSourceId, "coord_dec"].values)
)
plt.scatter(
    dra,
    ddec,
    c=np.log10(matches[found].forced_base_PsfFlux_instFlux_SNR),
    s=15,
    cmap="inferno",
    alpha=0.25,
)
plt.xlim(-0.6, 0.6)
plt.ylim(-0.6, 0.6)
plt.grid()
plt.title("Scatter of recovered fakes RA/Dec positions")
plt.xlabel(r"$\Delta RA$ [arcsec]", fontsize=14)
plt.ylabel(r"$\Delta Dec$ [arcsec]", fontsize=14)
plt.gca().set_aspect("equal")
plt.colorbar(label="SNR")

In [ ]:
dra = 3600 * (
    matches[found].ra
    - np.rad2deg(diaSrcs.loc[matches[found].diaSourceId, "coord_ra"].values)
)
ddec = 3600 * (
    matches[found].dec
    - np.rad2deg(diaSrcs.loc[matches[found].diaSourceId, "coord_dec"].values)
)
plt.scatter(
    dra,
    ddec,
    c=matches[found].mag,
    s=15,
    cmap="inferno",
    alpha=0.25,
)
plt.xlim(-0.6, 0.6)
plt.ylim(-0.6, 0.6)
plt.grid()
plt.title("Scatter of recovered fakes RA/Dec positions")
plt.xlabel(r"$\Delta RA$ [arcsec]", fontsize=14)
plt.ylabel(r"$\Delta Dec$ [arcsec]", fontsize=14)
plt.gca().set_aspect("equal")
plt.colorbar(label="mag")

In [ ]:
plt.hist(
    rbscores.loc[rbscores.diaSourceId.isin(matches[found].diaSourceId), "score"].values,
    log=True,
)
plt.xlabel("RB score")
plt.title("RB score for found fakes")

In [ ]:
matches_withRB = matches[matches.diaSourceId.isin(rbscores.index)]
plt.scatter(
    matches_withRB.dist_diaSrc,
    rbscores.loc[matches_withRB.diaSourceId, "score"].values,
    c=matches_withRB.mag,
    s=15,
    cmap="plasma",
    alpha=0.5,
)
plt.title("Recovered fake position offset vs Reliability score")
plt.xlabel(r"$\Delta r$ [arcsec]")
plt.ylabel("RB Score")
plt.colorbar(label="mag")

In [ ]:
# correlate SNR of fakes with RB score
plt.scatter(
    matches_withRB.forced_base_PsfFlux_instFlux_SNR,
    rbscores.loc[matches_withRB.diaSourceId, "score"].values,
    c=matches_withRB.mag,
    s=15,
    cmap="plasma",
    alpha=0.5,
)
# set log scale for SNR
plt.title("Recovered fake SNR vs Reliability score")
plt.xscale("log")
plt.xlabel("Fake SNR")
plt.ylabel("RB Score")
plt.colorbar(label="mag")

In [ ]:
plt.plot(
    matches[found].dist_diaSrc,
    matches[found].forced_base_PsfFlux_instFlux_SNR,
    ".",
    alpha=0.25,
)
plt.gca().set_yscale("log")
plt.gca().set_xscale("log")
plt.xlabel(r"$\Delta r$ [arcsec]")
plt.ylabel("Fake SNR")
plt.title("Recovered fake position offset vs SNR")
plt.grid()

In [ ]:
plt.plot(matches[found].ra, matches[found].dec, "bo", alpha=0.1, label="Found")
plt.plot(matches[~found].ra, matches[~found].dec, "r.", alpha=0.25, label="Lost")
plt.legend()
plt.gca().set_aspect(1 / np.cos(np.radians(-5)))
plt.grid()
plt.title("Scatter of fakes in sky coordinates")
plt.xlabel("R.A. [deg]")
plt.ylabel("Dec. [deg]")

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(matches[found].x, matches[found].y, "bo", alpha=0.05, label="Found")
plt.plot(matches[~found].x, matches[~found].y, "r.", alpha=0.25, label="Lost")
plt.legend(fontsize=10)
plt.gca().set_aspect(1)
plt.grid()
plt.title("Scatter of fakes in detector X/Y coordinates")
plt.xlabel("X [px]")
plt.ylabel("Y [px]")

### Efficiency calculation

In [ ]:
import matplotlib
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def power_law(x, k, a):
    """General power law model"""
    return k * (x**a)


def fsigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a * (x - b)))


def sigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(a * (x - b)))


def half_gaussian(x, sig, ampl, k):
    me = 0
    return ampl / (sig * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - me) / sig) ** 2)


def gaussian(x, sig, me, ampl):
    return ampl / (sig * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - me) / sig) ** 2)

In [ ]:
def estimate_effcurve(
    df, sncol="forced_base_PsfFlux_instFlux_SNR", matchedcol="found", wid=0.5, bins=None
):
    if bins is None:
        bins = np.arange(0, 50, wid)
    redges = bins[:-1]
    center = bins[:-1] + wid / 2.0

    counts, mbins = np.histogram(df[sncol].values, bins=bins)
    mcount, mbins = np.histogram(df[df[matchedcol] == True][sncol].values, bins=bins)

    eff = mcount / counts
    eff[np.where(counts == 0.0)] = 0.0
    err = 1.96 * np.sqrt(eff * (1 - eff) / counts)
    err[np.where(counts == 0.0)] = 1
    err[np.where(err == 0)] = 0.05

    popt, pcov = curve_fit(fsigmoid, center, eff, sigma=err, method="dogbox")
    eff_12 = np.round(popt[1], 2)

    hi_err = err.copy()
    hi_err[np.where((hi_err + eff) >= 1)] = (1 - eff)[np.where((hi_err + eff) >= 1)]
    yerr = [err, hi_err]
    return {
        "center": center,
        "fsigmoid": fsigmoid(center, popt[0], popt[1]),
        "redges": redges,
        "eff": eff,
        "eff_12": eff_12,
        "popt": popt,
        "pcov": pcov,
        "counts": counts,
        "mcounts": mcount,
        "bins": bins,
        "yerr": yerr,
        "hi_err": hi_err,
        "err": err,
        "mbins": mbins,
    }

In [ ]:
def plot_recall(df, ax=None, wid=0.5, bins=None, max_snr=16, **kwargs):
    if bins is None:
        bins = np.arange(0, 18, wid)
    results = estimate_effcurve(df, wid=wid, bins=bins, **kwargs)

    center = results["center"]
    fsigmd = results["fsigmoid"]
    redges = results["redges"]
    eff = results["eff"]
    eff_12 = results["eff_12"]
    popt = results["popt"]
    pcov = results["pcov"]
    counts = results["counts"]
    mcounts = results["mcounts"]
    bins = results["bins"]
    yerr = results["yerr"]
    hi_err = results["hi_err"]
    err = results["err"]

    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 4), nrows=1, ncols=1)

    ax.plot(center, fsigmd, "k-", label="Sigmoid fit", alpha=0.7)
    ax.step(redges, eff, "k", where="post", lw=1.5)
    ax.set_ylim(0, 1.1)
    ax.set_xlim(0, max_snr)

    ax.errorbar(redges + wid / 2.0, eff, yerr=yerr, fmt="k.")
    ax.axhline(y=0.5, xmin=0.05, xmax=0.95, color="gray", ls="-", lw=0.9)
    ax.axvline(
        x=eff_12,
        ymin=0,
        ymax=1.0,
        ls="--",
        label=r"$\rm{SNR}_{1/2} = $" + str(eff_12),
        color="k",
    )
    patches = []

    norm = colors.LogNorm(vmin=counts.min() + 1, vmax=counts.max())
    for abin, aneff, co in zip(redges, eff, counts):
        patch = Rectangle((abin, 0.0), wid, aneff)
        patches.append(patch)
    pcolors = norm(counts)
    pcolors[np.where(counts == 0.0)] = -1.0

    collection = PatchCollection(patches, cmap=plt.cm.gray, alpha=0.7)
    collection.set_array(pcolors)
    ax.add_collection(collection)

    ax.set_xlabel("Estimated truth SNR")
    ax.set_ylabel(r"Efficiency $\%$")
    ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
    ax.legend(loc="lower right")
    plt.colorbar(collection, ax=ax, label=r"$log_{10}(N)$")
    plt.tight_layout()
    plt.grid()
    return ax, results

#### Filtering fakes on bad locations

In [ ]:
matches_flags_when_false = [
    "forced_base_PixelFlags_flag_bad",
    "forced_base_LocalBackground_flag",
    "forced_base_PixelFlags_flag_interpolated",
    "forced_base_PixelFlags_flag_edgeCenter",
]

In [ ]:
filter_flags = np.ones(len(matches), dtype=bool)
print(filter_flags.sum())
for aflag in matches_flags_when_false:
    filter_flags &= ~matches[aflag].values
    print(aflag, filter_flags.sum())

In [ ]:
for aflt in matches_flags_when_false:
    print(f"{matches[aflt].sum():4}, {aflt}")

In [ ]:
bins = np.arange(-5, 100, 1)
plt.figure(figsize=(10, 6))
plt.hist(
    matches["forced_base_PsfFlux_instFlux_SNR"],
    bins=bins,
    histtype="step",
    label="No flag-filter",
)
plt.hist(
    matches[filter_flags]["forced_base_PsfFlux_instFlux_SNR"],
    bins=bins,
    histtype="step",
    label="flag-filtered",
)
plt.title("SNR distribution of filtered fakes")
plt.xlabel("SNR")
plt.legend(loc="best")
plt.show()

In [ ]:
bins = np.arange(16, 30, 0.25)
plt.figure(figsize=(10, 6))
plt.hist(matches["mag"], bins=bins, histtype="step", label="No flag-filter")
plt.hist(
    matches[filter_flags]["mag"], bins=bins, histtype="step", label="flag-filtered"
)
plt.xlabel("Fake mag")
plt.title("Magnitud distribution of filtered fakes")
plt.legend(loc="best")
plt.show()

In [ ]:
# plot xy for flag filtered sources
plt.figure(figsize=(10, 8))
plt.scatter(
    matches[~filter_flags].x,
    matches[~filter_flags].y,
    c=matches[~filter_flags].mag,
    cmap="plasma",
    vmin=18,
    vmax=30,
    s=matches[~filter_flags].mag,
)
plt.colorbar(label="mag")
plt.title("Flag-filtered fakes X/Y position")
plt.xlabel("X [px]")
plt.ylabel("Y [px]")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
plot_recall(matches, wid=0.5, ax=ax)
ax.set_xlim(0, 15)
ax.set_title("Efficiency for fakes SNR < 15 (all)")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax, results = plot_recall(matches[filter_flags], wid=0.5, ax=ax)
ax.set_title("Efficiency for *diaSrc matched* fakes SNR < 15 (flag-filtered)")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax, results = plot_recall(
    matches[filter_flags], wid=0.5, ax=ax, matchedcol="isCandidate"
)
ax.set_title("Efficiency for *candidateDiaSrc matched* fakes SNR < 15 (flag-filtered)")

In [ ]:
# Checking high SNR values
fig, ax = plt.subplots(1, 1, figsize=(18, 6))
ax, results = plot_recall(
    matches[filter_flags], wid=2, ax=ax, bins=np.arange(0, 150, 2), max_snr=150
)
ax.set_title("Efficiency for *diaSrc matched* SNR < 150")

In [ ]:
# Checking high SNR values
fig, ax = plt.subplots(1, 1, figsize=(18, 6))
ax, results = plot_recall(
    matches[filter_flags],
    wid=2,
    ax=ax,
    bins=np.arange(0, 150, 2),
    max_snr=150,
    matchedcol="isCandidate",
)
ax.set_title("Efficiency for *candidateDiaSrc matched* SNR < 150")

In [ ]:
# Checking high SNR values
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

hosted = matches[filter_flags & (matches["host_id"] > 10)]
hostless = matches[filter_flags & (matches["host_id"] < 10)]

ax1, results = plot_recall(
    hosted,
    wid=1,
    ax=ax1,
    bins=np.arange(0, 100, 1),
    max_snr=60,
    matchedcol="isCandidate",
)
ax2, results = plot_recall(
    hostless,
    wid=1,
    ax=ax2,
    bins=np.arange(0, 100, 1),
    max_snr=60,
    matchedcol="isCandidate",
)
ax1.set_title("*candidateDiaSrc matched* Hosted & SNR < 150")
ax2.set_title("*candidateDiaSrc matched* Host-*less* & SNR < 150")

In [ ]:
def plot_xy_eff(matches, found, foundcand, wid=100):
    # create x-y histograms and inspect trends
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8))
    bins = np.arange(0, 4200, wid)

    # add a left vertical axis to plot scatters
    ax12 = ax1.twinx()
    ax22 = ax2.twinx()

    ax12.set_ylabel("Efficiency %")
    ax22.set_ylabel("Efficiency %")
    ax22.yaxis.set_major_formatter(PercentFormatter(xmax=1))
    ax12.yaxis.set_major_formatter(PercentFormatter(xmax=1))

    ax1.set_title("X position")
    ax2.set_title("Y position")
    counts_x, binsx, _ = ax1.hist(
        matches.x, bins=bins, histtype="step", lw=2, label="All"
    )
    counts_y, binsy, _ = ax2.hist(
        matches.y, bins=bins, histtype="step", lw=2, label="All"
    )

    found_x, binsx, _ = ax1.hist(
        matches[found].x,
        bins=bins,
        histtype="step",
        lw=2,
        label="Found",
        alpha=0.8,
    )
    found_y, binsy, _ = ax2.hist(
        matches[found].y,
        bins=bins,
        histtype="step",
        lw=2,
        label="Found",
        alpha=0.8,
    )

    foundCand_x, binsx, _ = ax1.hist(
        matches[foundcand].x,
        bins=bins,
        histtype="stepfilled",
        lw=1,
        label="Found Candidate",
        color="gray",
        alpha=0.5,
    )
    foundCand_y, binsy, _ = ax2.hist(
        matches[foundcand].y,
        bins=bins,
        histtype="stepfilled",
        lw=1,
        label="Found Candidate",
        color="gray",
        alpha=0.5,
    )

    def get_eff(found_x, counts_x):
        effx = found_x / counts_x
        errx = 1.96 * np.sqrt(effx * (1 - effx) / counts_x)
        hi_errx = errx.copy()
        hi_errx[np.where((hi_errx + effx) >= 1)] = (1 - effx)[
            np.where((hi_errx + effx) >= 1)
        ]
        errx[np.where(errx == 0)] = 0.05
        err_x = [errx, hi_errx]
        return effx, err_x

    effx, err_x = get_eff(found_x, counts_x)
    effy, err_y = get_eff(found_y, counts_y)
    effCandx, errCand_x = get_eff(foundCand_x, counts_x)
    effCandy, errCand_y = get_eff(foundCand_y, counts_y)

    ax12.errorbar(binsx[:-1] + wid / 2, effx, err_x, c="k", label="diaSrc", alpha=0.65)
    ax22.errorbar(binsy[:-1] + wid / 2, effy, err_y, c="k", label="diaSrc", alpha=0.65)

    ax12.errorbar(
        binsx[:-1] + wid / 2,
        effCandx,
        errCand_x,
        c="k",
        label="candDiaSrc",
        alpha=0.5,
        linestyle="--",
    )
    ax22.errorbar(
        binsy[:-1] + wid / 2,
        effCandy,
        errCand_y,
        c="k",
        label="candDiaSrc",
        alpha=0.5,
        linestyle="--",
    )
    plt.tight_layout()

    return fig

In [ ]:
fig = plot_xy_eff(matches, found, matches["isCandidate"], wid=100)
fig.suptitle("Distribution of X/Y position. No flag-filter.")
plt.tight_layout()

In [ ]:
fig = plot_xy_eff(
    matches[filter_flags],
    matches[filter_flags]["found"],
    matches[filter_flags]["isCandidate"],
    wid=100,
)
fig.suptitle("Distribution of X/Y position. Flag filtered fakes.")
plt.tight_layout()

In [ ]:
hi_snr = matches[(matches.forced_base_PsfFlux_instFlux_SNR > 5) & filter_flags]

fig = plot_xy_eff(hi_snr, hi_snr["found"], hi_snr["isCandidate"], wid=100)
fig.suptitle("SNR > 5 & Flag filtered fakes")
plt.tight_layout()

In [ ]:
# efficiency as function of any other parameter
def get_efficiency(
    df,
    xcol="forced_base_PsfFlux_instFlux_SNR",
    foundcol="found",
    bins=None,
    wid=0.5,
    min_snr=5,
):
    if bins is None:
        bins = np.arange(np.nanmin(df[xcol]), np.nanmax(df[xcol]), wid)

    redges = bins[:-1]
    center = bins[:-1] + wid / 2.0

    counts, mbins = np.histogram(df[xcol].values, bins=bins)
    mcount, mbins = np.histogram(df[df[foundcol] == True][xcol].values, bins=bins)

    eff = mcount / counts
    eff[np.where(counts == 0.0)] = 0.0
    err = 1.96 * np.sqrt(eff * (1 - eff) / counts)
    err[np.where(counts == 0.0)] = 0.1
    err[np.where(err == 0)] = 0.005

    hi_err = err.copy()
    hi_err[np.where((hi_err + eff) >= 1)] = (1 - eff)[np.where((hi_err + eff) >= 1)]
    yerr = [err, hi_err]

    return {
        "center": center,
        "redges": redges,
        "eff": eff,
        "counts": counts,
        "mcounts": mcount,
        "bins": bins,
        "yerr": yerr,
        "hi_err": hi_err,
        "err": err,
        "mbins": mbins,
    }

In [ ]:
def plot_general_eff(df, xcol, bins, wid, foundcol="found", candidateCol="isCandidate"):
    results = get_efficiency(df, xcol=xcol, foundcol=foundcol, wid=wid, bins=bins)
    resultsCand = get_efficiency(
        df, xcol=xcol, foundcol=candidateCol, wid=wid, bins=bins
    )

    fig, ax1 = plt.subplots(1, 1, figsize=(12, 6))

    # add a left vertical axis to plot scatters
    ax12 = ax1.twinx()
    ax12.grid()

    ax12.set_ylabel("Efficiency %")
    ax12.yaxis.set_major_formatter(PercentFormatter(xmax=1))
    ax12.set_ylim(0, 1.18)

    ax1.set_title(xcol)
    counts_x, binsx, _ = ax1.hist(
        df[xcol], bins=results["bins"], histtype="step", lw=2, label="All Fakes"
    )
    found_x, binsx, _ = ax1.hist(
        df[df[foundcol]][xcol],
        bins=results["bins"],
        histtype="step",
        lw=2,
        label="Found diaSrc",
        alpha=0.8,
    )
    cand_x, binsx, _ = ax1.hist(
        df[df[candidateCol]][xcol],
        bins=resultsCand["bins"],
        histtype="stepfilled",
        lw=1,
        label="Found candDiaSrc",
        color="gray",
        alpha=0.5,
    )

    ax12.errorbar(
        binsx[:-1] + wid / 2,
        results["eff"],
        results["yerr"],
        c="k",
        label="diaSrc",
        alpha=0.75,
    )
    ax12.errorbar(
        binsx[:-1] + wid / 2,
        resultsCand["eff"],
        resultsCand["yerr"],
        c="k",
        label="candDiaSrc",
        alpha=0.5,
        linestyle="--",
    )

    ax1.set_xlabel(xcol)
    ax1.set_ylabel("N fakes")
    ax1.legend(loc="best", fontsize=10)
    plt.tight_layout()

    return ax1, results

### Parameters as predictors of detection

In [ ]:
xcol = "forced_base_PsfFlux_instFlux_SNR"
wid = 0.5
minx = 0
maxx = 30
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(matches[filter_flags], xcol, bins, wid)
ax.set_title("SNR of fakes- flag-filtered")

In [ ]:
xcol = "mag"
foundcol = "found"
wid = 0.25
minx = 18
maxx = 28
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(matches[filter_flags], xcol, bins, wid)
ax.set_title("mag of fakes- flag-filtered")

### From this point on we filter on SNR > 5

In [ ]:
# Filtering the low SNR from now on:
matches_hiSNR = matches[
    (matches["forced_base_PsfFlux_instFlux_SNR"] > 5) & filter_flags
]
foundcol = "found"

In [ ]:
xcol = "delta_mag"
wid = 0.21
minx = -0.1
maxx = 2
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(
    matches_hiSNR[matches_hiSNR["host_id"] > 1], xcol, bins, wid
)
ax.set_title("$mag - mag_{host}$")

In [ ]:
xcol = "dist_host"
wid = 0.1
minx = 0
maxx = 1.5
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(
    matches_hiSNR[matches_hiSNR["host_id"] > 1], xcol, bins, wid
)
ax.set_title("Distance to host $\Delta r_{host}$")

In [ ]:
xcol = "host_magnitude"
wid = 0.3
minx = 15
maxx = 27
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(matches_hiSNR, xcol, bins, wid)
ax.set_title("Host magnitude $mag_{host}$")

In [ ]:
xcol = "forced_base_PsfFlux_area"
wid = 5
minx = 60
maxx = 190
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(matches_hiSNR, xcol, bins, wid)
ax.set_title("$PSF_{area}$")

In [ ]:
xcol = "forced_base_PsfFlux_chi2"
wid = 10
minx = 1300
maxx = 2100
bins = np.linspace(minx, maxx, 30)
wid = bins[1] - bins[0]
ax, results = plot_general_eff(matches_hiSNR, xcol, bins, wid)
ax.set_title("PSF $\chi^2$")

In [ ]:
xcol = "delta_dec"
wid = 0.2 / 3600
minx = 0
maxx = 2.0 / 3600
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(matches_hiSNR, xcol, bins, wid)

In [ ]:
xcol = "delta_ra"
bins = np.arange(minx, maxx, wid)
ax, results = plot_general_eff(matches_hiSNR, xcol, bins, wid)

In [ ]:
xcol = "injection_draw_size"
wid = 2
minx = 25
maxx = 36
bins = np.arange(minx, maxx, wid)

ax, results = plot_general_eff(matches_hiSNR, xcol, bins, wid)

### Flux metrics

In [ ]:
merged_fluxes = pd.merge(
    left=matches_hiSNR,
    right=diaSrcs,
    left_on="diaSourceId",
    right_on="id",
    how="left",
    suffixes=("_ssi", "_diasrc"),
)

In [ ]:
merged_fluxes["base_PsfFlux_mag"] = (
    (u.nanojansky * merged_fluxes["base_PsfFlux_instFlux"].values).to(u.ABmag).value
)
merged_fluxes["base_PsfFlux_magErr"] = 1 / (
    merged_fluxes["base_PsfFlux_instFlux"] / merged_fluxes["base_PsfFlux_instFluxErr"]
)
merged_fluxes["slot_ApFlux_mag"] = (
    (u.nanojansky * merged_fluxes["slot_ApFlux_instFlux"].values).to(u.ABmag).value
)
merged_fluxes["slot_ApFlux_magErr"] = 1 / (
    merged_fluxes["slot_ApFlux_instFlux"] / merged_fluxes["slot_ApFlux_instFluxErr"]
)

In [ ]:
bins = np.arange(16, 30, 0.25)
plt.hist(
    merged_fluxes["base_PsfFlux_mag"],
    histtype="step",
    lw=2,
    bins=bins,
    label="PSF mags",
)
plt.hist(
    merged_fluxes["slot_ApFlux_mag"],
    histtype="step",
    lw=2,
    alpha=0.5,
    bins=bins,
    label="Aper mag",
)
plt.legend(loc="best", fontsize=10)
plt.title("diaSrc Photometry on fakes")

In [ ]:
bins = np.arange(-1, 150, 1)
plt.hist(
    1.0 / merged_fluxes["base_PsfFlux_magErr"],
    histtype="step",
    lw=2,
    bins=bins,
    label="PSF mags",
)
plt.hist(
    1.0 / merged_fluxes["slot_ApFlux_magErr"],
    histtype="step",
    lw=2,
    alpha=0.5,
    bins=bins,
    label="Aper mags",
)
plt.legend(loc="best")
plt.xlabel("SNR")
plt.title("diaSrc Photometry on fakes")
plt.show()

In [ ]:
bins = np.arange(-0.1, 1.5, 0.01)
plt.hist(
    merged_fluxes["base_PsfFlux_magErr"],
    histtype="step",
    lw=2,
    bins=bins,
    label="PSF mags",
)
plt.hist(
    merged_fluxes["slot_ApFlux_magErr"],
    histtype="step",
    lw=2,
    alpha=0.5,
    bins=bins,
    label="Aper mags",
)
plt.legend(loc="best")
plt.title("diaSrc Photometry on fakes")
plt.xlabel("mag errors")
plt.show()

In [ ]:
from astropy.stats import sigma_clipped_stats
from scipy import stats

In [ ]:
def binned_median_scatter(
    x,
    y,
    xbins=None,
    ax=None,
    label=None,
    error_in_mean=False,
    medians_args={"markersize": 5, "color": "k", "marker": "o"},
    *args,
    **kwargs
):
    """Produce a scatter plot and binning using the median"""

    if "alpha" not in kwargs:
        kwargs["alpha"] = 0.2

    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    if label is None:
        label = "Data"

    if xbins is None:
        finite = np.isfinite(x)
        xmax = np.nanmax(x[finite])
        xmin = np.nanmin(x[finite])
        nx = len(x)
        binsize = 5 * (xmax - xmin) / np.sqrt(nx)
        xbins = np.arange(xmin, xmax, binsize)

    xx = np.asarray(x[np.isfinite(x) & np.isfinite(y)])
    yy = np.asarray(y[np.isfinite(x) & np.isfinite(y)])

    bincenters = (xbins[:-1] + xbins[1:]) / 2
    ax.scatter(xx, yy, s=3, label=label, *args, **kwargs)

    clp_median = lambda x: sigma_clipped_stats(x)[1]
    clp_std = lambda x: sigma_clipped_stats(x)[2]

    means, bin_edges, ibins = stats.binned_statistic(
        xx, yy, bins=xbins, statistic=clp_median
    )
    stds, _, _ = stats.binned_statistic(xx, yy, bins=xbins, statistic=clp_std)
    counts, _, _ = stats.binned_statistic(xx, yy, bins=xbins, statistic="count")

    ff = counts > 15
    if error_in_mean:
        errors = stds[ff] / np.sqrt(counts[ff])
    else:
        errors = stds[ff]

    if len(medians_args) != 0:
        ax.errorbar(
            bincenters[ff],
            means[ff],
            errors,
            xerr=(xbins[1] - xbins[0]) / 2,
            **medians_args,
        )
    else:
        ax.errorbar(
            bincenters[ff],
            means[ff],
            errors,
            xerr=(xbins[1] - xbins[0]) / 2,
            color="k",
            fmt=".",
            label="",
        )

    return ax

In [ ]:
xbins = np.arange(17, 24, 0.25)
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax = binned_median_scatter(
    merged_fluxes["mag"],
    merged_fluxes["mag"].values - merged_fluxes["base_PsfFlux_mag"].values,
    alpha=0.5,
    xbins=xbins,
    ax=ax,
    error_in_mean=False,
    color="gray",
    medians_args={
        "markersize": 6,
        "color": "k",
        "marker": "o",
        "alpha": 0.95,
        "markerfacecolor": "tab:gray",
        "markerfacecoloralt": "gray",
        "markeredgecolor": "k",
    },
)
ax.set_ylabel("Fake mag - PSF mag", fontsize=14)
ax.set_xlabel("Fake mag", fontsize=14)
ax.set_ylim(-0.2, 0.25)
plt.title("PSF mag offsets")
ax.grid()
plt.tight_layout()

In [ ]:
xbins = np.arange(17, 24, 0.25)
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax = binned_median_scatter(
    merged_fluxes["mag"],
    merged_fluxes["mag"].values - merged_fluxes["slot_ApFlux_mag"].values,
    alpha=0.5,
    xbins=xbins,
    ax=ax,
    error_in_mean=False,
    color="gray",
    medians_args={
        "markersize": 6,
        "color": "k",
        "marker": "o",
        "alpha": 0.95,
        "markerfacecolor": "tab:gray",
        "markerfacecoloralt": "gray",
        "markeredgecolor": "k",
    },
)
ax.set_ylabel("Fake mag - Ap mag", fontsize=14)
ax.set_xlabel("Fake mag", fontsize=14)
ax.set_ylim(-0.2, 0.25)
plt.title("Ap mag offsets")
ax.grid()
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(*gridplots, figsize=(10, 6))
iax = 0
xbins = np.arange(17, 24, 0.25)

fig.suptitle("PSF mag offsets")
for flt in bands:
    ax = axes.flatten()[iax] if len(bands) > 1 else axes
    ax.set_title(f"Fakes in LSSTCam {flt} filter")
    tab = merged_fluxes[merged_fluxes["band_ssi"] == flt]
    ax = binned_median_scatter(
        tab["mag"],
        tab["mag"].values - tab["base_PsfFlux_mag"].values,
        alpha=0.5,
        xbins=xbins,
        ax=ax,
        error_in_mean=False,
        color=clrs[flt],
        medians_args={
            "markersize": 6,
            "color": "k",
            "marker": "o",
            "alpha": 0.95,
            "markerfacecolor": clrs[flt],
            "markerfacecoloralt": "lightsteelblue",
            "markeredgecolor": "k",
        },
    )
    ax.set_ylabel("Fake mag - PSF mag", fontsize=14)
    ax.set_xlabel("Fake mag", fontsize=14)
    ax.set_ylim(-0.2, 0.25)

    ax.grid()
    iax += 1

In [ ]:
fig, axes = plt.subplots(*gridplots, figsize=(10, 6))
iax = 0
xbins = np.arange(17, 24, 0.25)
fig.suptitle("Ap mag offsets")

for flt in bands:
    ax = axes.flatten()[iax] if len(bands) > 1 else axes
    ax.set_title(f"Fakes in LSSTCam {flt} filter")
    tab = merged_fluxes[merged_fluxes["band_ssi"] == flt]
    ax = binned_median_scatter(
        tab["mag"],
        tab["mag"].values - tab["slot_ApFlux_mag"].values,
        alpha=0.5,
        xbins=xbins,
        ax=ax,
        error_in_mean=False,
        color=clrs[flt],
        medians_args={
            "markersize": 6,
            "color": "k",
            "marker": "o",
            "alpha": 0.95,
            "markerfacecolor": clrs[flt],
            "markerfacecoloralt": "lightsteelblue",
            "markeredgecolor": "k",
        },
    )
    ax.set_ylabel("Fake mag - Ap mag", fontsize=14)
    ax.set_xlabel("Fake mag", fontsize=14)
    ax.set_ylim(-0.2, 0.25)
    ax.grid()
    iax += 1

In [ ]:
fig, axes = plt.subplots(*gridplots, figsize=(10, 6))
iax = 0
fig.suptitle("Fake mag - PSF mag vs matching distance", fontsize=14)
xbins = np.arange(0, 0.25, 0.02)

for flt in bands:
    ax = axes.flatten()[iax] if len(bands) > 1 else axes
    ax.set_title(f"Fakes in LSSTCam {flt} filter")
    tabmatches = merged_fluxes[merged_fluxes["band_ssi"] == flt]
    ax = binned_median_scatter(
        tabmatches["dist_diaSrc"],
        tabmatches["mag"].values - tabmatches["base_PsfFlux_mag"].values,
        xbins=xbins,
        ax=ax,
        error_in_mean=False,
        alpha=0.6,
        color=clrs[flt],
        medians_args={
            "markersize": 7,
            "color": "k",
            "marker": "o",
            "alpha": 0.75,
            "markerfacecolor": clrs[flt],
            "markerfacecoloralt": "lightsteelblue",
            "markeredgecolor": "k",
        },
    )
    ax.grid()
    ax.set_ylabel("Fake mag - PSF mag", fontsize=12)
    ax.set_xlabel(r"$\Delta r$ [arcsec]")
    ax.set_ylim(-0.3, 0.35)
    ax.set_xlim(0, 0.35)
    iax += 1

In [ ]:
fig, axes = plt.subplots(*gridplots, figsize=(10, 6))
iax = 0
fig.suptitle("Fake mag - Ap mag vs matching distance", fontsize=14)
xbins = np.arange(0, 0.25, 0.02)

for flt in bands:
    ax = axes.flatten()[iax] if len(bands) > 1 else axes
    ax.set_title(f"Fakes in LSSTCam {flt} filter")
    tabmatches = merged_fluxes[merged_fluxes["band_ssi"] == flt]
    ax = binned_median_scatter(
        tabmatches["dist_diaSrc"],
        tabmatches["mag"].values - tabmatches["slot_ApFlux_mag"].values,
        xbins=xbins,
        ax=ax,
        error_in_mean=False,
        alpha=0.6,
        color=clrs[flt],
        medians_args={
            "markersize": 7,
            "color": "k",
            "marker": "o",
            "alpha": 0.75,
            "markerfacecolor": clrs[flt],
            "markerfacecoloralt": "lightsteelblue",
            "markeredgecolor": "k",
        },
    )
    ax.grid()
    ax.set_ylabel("Fake mag - Ap mag", fontsize=12)
    ax.set_xlabel(r"$\Delta r$ [arcsec]")
    ax.set_ylim(-0.3, 0.35)
    ax.set_xlim(0, 0.35)
    iax += 1

In [ ]:
merged_fluxes["pulls_psf"] = (
    merged_fluxes["base_PsfFlux_instFlux"]
    - (merged_fluxes["mag"].values * u.ABmag).to(u.nanojansky).value
) / merged_fluxes["base_PsfFlux_instFluxErr"]
merged_fluxes["pulls_ap"] = (
    merged_fluxes["slot_ApFlux_instFlux"]
    - (merged_fluxes["mag"].values * u.ABmag).to(u.nanojansky).value
) / merged_fluxes["slot_ApFlux_instFluxErr"]

In [ ]:
norm_wid = 0.1
datawid = 0.2
norm_bins = np.arange(-8.25, 8, norm_wid)
bins = np.arange(-8.25, 8, datawid)
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
pulls = merged_fluxes[
    (merged_fluxes["pulls_psf"] < 8) & (merged_fluxes["pulls_psf"] > -8)
]
psfmean, psfmed, psfstd = sigma_clipped_stats(pulls["pulls_psf"])
apmean, apmed, apstd = sigma_clipped_stats(pulls["pulls_ap"])

ax.plot(
    norm_bins,
    stats.norm.pdf(norm_bins) * norm_wid * len(pulls) * datawid / norm_wid,
    color="k",
    lw=1.5,
)

ax.hist(
    pulls["pulls_psf"].values,
    bins=bins,
    histtype="step",
    lw=2,
    color="k",
    label=f"PSF mag\n$\mu={psfmed:3.2f}$\n$\sigma={psfstd:3.2f}$",
)
ax.hist(
    pulls["pulls_ap"].values,
    bins=bins,
    histtype="stepfilled",
    lw=2,
    color="gray",
    linestyle="--",
    alpha=0.5,
    label=f"AP mag\n$\mu={apmed:3.2f}$\n$\sigma={apstd:3.2f}$",
)
ax.legend(fontsize=12)
plt.title("Pull distribution for PSF fluxes - all bands")
plt.show()

In [ ]:
fig, axes = plt.subplots(*gridplots, figsize=(10, 6))
iax = 0
fig.suptitle("Pulls distribution for PSF fluxes per band", fontsize=14)
norm_wid = 0.1
datawid = 0.2
norm_bins = np.arange(-8.25, 8, norm_wid)
bins = np.arange(-8.25, 8, datawid)

for flt in bands:
    ax = axes.flatten()[iax] if len(bands) > 1 else axes
    ax.set_title(f"Pulls fakes in LSSTCam {flt} filter")
    tabmatches = merged_fluxes[merged_fluxes["band_ssi"] == flt]
    pulls = tabmatches[(tabmatches["pulls_psf"] < 8) & (tabmatches["pulls_psf"] > -8)]
    psfmean, psfmed, psfstd = sigma_clipped_stats(pulls["pulls_psf"])
    apmean, apmed, apstd = sigma_clipped_stats(pulls["pulls_ap"])

    ax.plot(
        norm_bins,
        stats.norm.pdf(norm_bins) * norm_wid * len(pulls) * datawid / norm_wid,
        color="k",
        lw=1.5,
    )

    ax.hist(
        pulls["pulls_psf"].values,
        bins=bins,
        histtype="step",
        lw=2,
        color=clrs[flt],
        label=f"PSF mag\n$\mu={psfmed:3.2f}$\n$\sigma={psfstd:3.2f}$",
    )
    ax.hist(
        pulls["pulls_ap"].values,
        bins=bins,
        histtype="stepfilled",
        lw=2,
        color=clrs[flt],
        linestyle="--",
        alpha=0.25,
        label=f"AP mag\n$\mu={apmed:3.2f}$\n$\sigma={apstd:3.2f}$",
    )
    ax.legend(fontsize=12)
plt.tight_layout()
plt.show()